In [1]:
# Package telepítés
import sys

print("📦 Csomagok telepítése...")
%pip install -q --upgrade pip
%pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
%pip install -q transformers datasets huggingface_hub
%pip install -q numpy scipy scikit-learn pandas
%pip install -q accelerate peft bitsandbytes
%pip install -q --upgrade trl unsloth
%pip install -q sentence-transformers faiss-gpu bm25s ranx
%pip install -q tqdm

print("✅ Csomagok telepítve")


📦 Csomagok telepítése...
/Users/zelenyianszkimate/Documents/CourtRankRL/.venv/bin/python: No module named pip
/Users/zelenyianszkimate/Documents/CourtRankRL/.venv/bin/python: No module named pip
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
/Users/zelenyianszkimate/Documents/CourtRankRL/.venv/bin/python: No module named pip
/Users/zelenyianszkimate/Documents/CourtRankRL/.venv/bin/python: No module named pip
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
/Users/zelenyianszkimate/Documents/CourtRankRL/.venv/bin/python: No module named pip
/Users/zelenyianszkimate/Documents/CourtRankRL/.venv/bin/python: No module named pip
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
/Users/zelenyianszkimate/Documents/CourtRankRL/.venv/bin/python: No m

In [2]:
# Importok
from unsloth import FastLanguageModel

import os
import json
import warnings
from pathlib import Path
from typing import Dict, List, Tuple, Optional
from collections import Counter, defaultdict

import numpy as np
import pandas as pd
import torch
from tqdm.auto import tqdm

# ML libraries
from sklearn.metrics import ndcg_score
from scipy.stats import entropy as scipy_entropy

# Retrieval libraries
import bm25s
import faiss
from sentence_transformers import SentenceTransformer

# Evaluation
from ranx import Qrels, Run, evaluate

# Model libraries
from huggingface_hub import login

warnings.filterwarnings("ignore")
os.environ["TOKENIZERS_PARALLELISM"] = "false"

print("✅ Importok betöltve")
print(f"PyTorch verzió: {torch.__version__}")
print(f"CUDA elérhető: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU memória: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")


ModuleNotFoundError: No module named 'unsloth'

In [ ]:
# Útvonalak
BASE_PATH = Path("/workspace")
MODEL_PATH = BASE_PATH / "grpo_policy"
EVAL_SPLIT_PATH = BASE_PATH / "eval_split.json"
QRELS_PATH = BASE_PATH / "baseline_qrels.tsv"
BASELINE_METRICS_PATH = BASE_PATH / "baseline_query_metrics.csv"
CHUNKS_PATH = BASE_PATH / "chunks.jsonl"
FAISS_INDEX_PATH = BASE_PATH / "faiss_index.bin"
CHUNK_ID_MAP_PATH = BASE_PATH / "chunk_id_map.npy"
BM25_INDEX_PATH = BASE_PATH / "bm25_index" / "bm25s_model"
CHUNK_IDS_PATH = BASE_PATH / "chunk_ids.json"
OUTPUT_DIR = BASE_PATH / "evaluation_results"

OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Konfiguráció
SLATE_SIZE = 30
EVAL_TOP_K = 20
TOP_K_BASELINE = 300
RRF_K = 60
MAX_SEQ_LENGTH = 16384
MAX_NEW_TOKENS = 768
TEMPERATURE = 0.8
TOP_P = 0.9

print("📋 Konfiguráció:")
print(f"  Base path: {BASE_PATH}")
print(f"  Model path: {MODEL_PATH}")
print(f"  Output dir: {OUTPUT_DIR}")
print(f"  Slate size: {SLATE_SIZE}")
print(f"  ⚠️  Eval top-k: {EVAL_TOP_K} (truncated for fair baseline comparison)")
print(f"  Max sequence length: {MAX_SEQ_LENGTH}")

# Fájlellenőrzés
print("\n📂 Fájlellenőrzés:")
required_files = [
    (MODEL_PATH, "Model directory"),
    (EVAL_SPLIT_PATH, "Eval split"),
    (QRELS_PATH, "Qrels"),
    (BASELINE_METRICS_PATH, "Baseline metrics"),
    (CHUNKS_PATH, "Chunks"),
    (FAISS_INDEX_PATH, "FAISS index"),
    (CHUNK_ID_MAP_PATH, "Chunk ID map"),
    (BM25_INDEX_PATH, "BM25 index"),
    (CHUNK_IDS_PATH, "Chunk IDs"),
]

all_files_exist = True
for path, name in required_files:
    if path.exists():
        print(f"  ✅ {name}: {path}")
    else:
        print(f"  ❌ {name}: HIÁNYZIK - {path}")
        all_files_exist = False

if not all_files_exist:
    raise FileNotFoundError("❌ Hiányzó fájlok! Töltsd fel az összes szükséges fájlt.")


print("\n✅ Minden szükséges fájl elérhető")

In [ ]:
# === METRIKA HELPER FÜGGVÉNYEK ===

def calculate_ndcg(ranked_indices: List[int], true_relevance: List[float], k: int = 10) -> float:
    """nDCG számítás sklearn-kel."""
    if not true_relevance or not ranked_indices or max(true_relevance) == 0:
        return 0.0
    y_true = np.array(true_relevance)
    max_score = len(ranked_indices)
    y_score = np.zeros_like(y_true, dtype=float)
    for i, idx in enumerate(ranked_indices[:k]):
        if idx < len(y_true):
            y_score[idx] = max_score - i
    if np.sum(y_score) == 0:
        return 0.0
    try:
        return float(ndcg_score(y_true.reshape(1, -1), y_score.reshape(1, -1), k=k))
    except:
        return 0.0

def compute_ndcg_metric(relevances: List[int], k: int) -> float:
    """nDCG@k számítás ranking sorrendből."""
    if not relevances or k <= 0:
        return 0.0
    relevances_truncated = relevances[:k]
    scores = list(range(len(relevances_truncated), 0, -1))
    try:
        ndcg = ndcg_score(
            y_true=[relevances_truncated],
            y_score=[scores],
            k=k
        )
        return float(ndcg)
    except Exception:
        return 0.0

def compute_mrr(relevances: List[int], k: Optional[int] = None) -> float:
    """Mean Reciprocal Rank számítás."""
    if k is not None:
        relevances = relevances[:k]
    for i, rel in enumerate(relevances, start=1):
        if rel > 0:
            return 1.0 / i
    return 0.0

def compute_recall(relevances: List[int], k: int, total_relevant: Optional[int] = None) -> float:
    """Recall@k számítás."""
    if not relevances:
        return 0.0
    relevant_in_topk = sum(1 for rel in relevances[:k] if rel > 0)
    if total_relevant is None:
        total_relevant = sum(1 for rel in relevances if rel > 0)
    if total_relevant == 0:
        return 0.0
    return relevant_in_topk / total_relevant

print("✅ Metrika helper függvények definiálva")


In [ ]:
# === PARSING ÉS PROMPT FÜGGVÉNYEK ===

def parse_model_ranking(completion: str, slate_size: int) -> List[int]:
    """
    Model completion parsing - kinyeri a ranking indexeket.
    Ha nincs elég valid index, kiegészíti a baseline order szerint.
    """
    try:
        numbers = []
        
        # 1. Vesszővel elválasztott számok
        parts = completion.split(",")
        for part in parts:
            part = part.strip()
            for word in part.split():
                word = word.strip().rstrip(".,;")
                if word.isdigit():
                    num = int(word)
                    if 0 <= num < slate_size:
                        numbers.append(num)
        
        # 2. Ha nincs elég, próbálunk szóközzel elválasztott formátumot
        if len(numbers) < slate_size // 3:
            for word in completion.split():
                word = word.strip().rstrip(".,;")
                if word.isdigit():
                    num = int(word)
                    if 0 <= num < slate_size and num not in numbers:
                        numbers.append(num)
        
        # 3. Valid számok gyűjtése (deduplikáció)
        valid_numbers = []
        seen = set()
        for n in numbers:
            if n not in seen:
                valid_numbers.append(n)
                seen.add(n)
        
        # 4. Ha van elég valid szám, használjuk
        if len(valid_numbers) >= max(1, slate_size // 3):
            missing_indices = [i for i in range(slate_size) if i not in valid_numbers]
            result = valid_numbers + missing_indices
            return result[:slate_size]
        
    except Exception as e:
        pass
    
    # Fallback: baseline order
    return list(range(slate_size))

def create_evaluation_prompt(query_id: str, slate: List[Dict]) -> str:
    """
    🔒 BLIND EVALUATION prompt - NO relevance labels (data leakage fix!)
    
    CRITICAL DIFFERENCE FROM TRAINING:
    - Training prompt (grpo_train_runpod.ipynb): Includes relevance labels for reward calculation
    - Evaluation prompt (THIS): NO relevance labels - blind assessment only
    
    The model must rank purely based on:
    - Document content
    - Query-document semantic match
    - Legal domain relevance
    - BM25/FAISS signals (optional)
    
    Relevance scores are ONLY used post-hoc for metric calculation, NOT shown to model!
    """
    prompt = f'''# Legal Document Relevance Assessment and Ranking

## TASK
You are helping to rank Hungarian court decision documents by their relevance to a search query.

## SEARCH QUERY
"{query_id}"

## WHAT TO CONSIDER

When ranking documents, think about:
- How well the document topic matches the query
- Whether the content directly addresses what the query is asking
- The legal domain and court type (may or may not be relevant)
- BM25 and FAISS scores show retrieval system confidence, but content is most important

## DOCUMENT EVALUATION

You need to evaluate the following {len(slate)} document excerpts:

'''
    
    separator_line = "\u2500" * 63
    
    for idx, doc in enumerate(slate):
        chunk_text = doc.get('text', '')[:800]
        bm25 = doc.get('bm25_score', 0)
        faiss = doc.get('faiss_score', 0)
        court = doc.get('court', 'N/A')
        domain = doc.get('domain', 'N/A')
        year = doc.get('year', 'N/A')
        
        # 🔒 CRITICAL: NO relevance label in evaluation prompt!
        # relevance field exists in slate for metrics calculation only
        
        prompt += f'''{separator_line}
DOCUMENT [{idx}]

ID: {doc.get('doc_id', 'N/A')} | Court: {court} | Domain: {domain} | Year: {year}
Scores: BM25={bm25:.2f}, FAISS={faiss:.3f}

Content:
{chunk_text}

'''
    
    prompt += f'''{separator_line}

## RANKING INSTRUCTIONS

You need to rank the {len(slate)} documents by their relevance to the search query.

### GUIDELINES (focus on relevance):
- Consider the search query and how well each document matches it
- Analyze document content, topic match, and legal domain relevance
- BM25 and FAISS scores show retrieval system confidence, but focus primarily on actual content relevance to the query
- Use your judgment to determine which documents best answer the legal query

### OUTPUT FORMAT (STRICT):
- Respond with EXACTLY one line in this format → RANKING: <comma-separated indices> #END
- Use document indices 0 to {len(slate)-1}, each exactly once, separated by commas and without extra spaces
- Do NOT include headings, bullet points, Markdown, or explanations before or after the line
- If you are uncertain, still provide your best full ranking covering every index
- End the line with "#END" to confirm completion

Example:
RANKING: 3,0,5,1,4,2,8,6,7,9 #END

RANKING:
'''
    
    return prompt

print("✅ Parsing és prompt függvények definiálva")
print("   🔒 BLIND EVALUATION MODE: Relevance labels REMOVED from prompt!")


In [ ]:
# === EVAL SPLIT BETÖLTÉSE ===
print("📂 Eval split betöltése...")
with open(EVAL_SPLIT_PATH, 'r', encoding='utf-8') as f:
    eval_split_data = json.load(f)

eval_query_ids = eval_split_data.get("query_ids", [])
split_params = eval_split_data.get("split_params", {})

print(f"✅ Eval split betöltve: {len(eval_query_ids)} query")
print(f"   Split paraméterek: seed={split_params.get('seed')}, train_split={split_params.get('train_split')}")


In [ ]:
# === QRELS BETÖLTÉSE ===
print("\n📂 Qrels betöltése...")
qrels_dict = {}
with open(QRELS_PATH, 'r', encoding='utf-8') as f:
    lines = f.readlines()
    for line in lines[1:]:  # Skip header
        line = line.strip()
        if not line:
            continue
        parts = line.split('\t')
        if len(parts) != 3:
            continue
        query_id, doc_id, relevance = parts
        if query_id not in qrels_dict:
            qrels_dict[query_id] = {}
        qrels_dict[query_id][doc_id] = int(relevance)

print(f"✅ Qrels betöltve: {len(qrels_dict)} query, {sum(len(docs) for docs in qrels_dict.values())} relevancia判定")

# Relevancia eloszlás
relevance_counts = defaultdict(int)
for query_rels in qrels_dict.values():
    for rel_grade in query_rels.values():
        relevance_counts[rel_grade] += 1

print("\n📊 Relevancia eloszlás:")
for grade in sorted(relevance_counts.keys()):
    print(f"  Grade {grade}: {relevance_counts[grade]} dokumentum")


In [ ]:
# === BASELINE METRIKÁK BETÖLTÉSE ===
print("\n📂 Baseline metrikák betöltése...")
baseline_df = pd.read_csv(BASELINE_METRICS_PATH, encoding='utf-8')
baseline_metrics_dict = {
    row["query_id"]: row.to_dict()
    for _, row in baseline_df.iterrows()
}

print(f"✅ Baseline metrikák betöltve: {len(baseline_metrics_dict)} query")

# Aggregált baseline teljesítmény
avg_ndcg10 = np.mean([m.get("ndcg@10", 0.0) for m in baseline_metrics_dict.values()])
avg_mrr = np.mean([m.get("mrr", 0.0) for m in baseline_metrics_dict.values()])
avg_map = np.mean([m.get("map", 0.0) for m in baseline_metrics_dict.values()])

print(f"\n🎯 Baseline teljesítmény (átlag):")
print(f"  • nDCG@10: {avg_ndcg10:.4f}")
print(f"  • MRR:     {avg_mrr:.4f}")
print(f"  • MAP:     {avg_map:.4f}")


In [ ]:
# === CHUNKS BETÖLTÉSE ===
print("\n📂 Chunks betöltése...")
chunks_df = pd.read_json(CHUNKS_PATH, lines=True, encoding='utf-8')

print(f"✅ Chunks betöltve: {len(chunks_df):,} chunk")
print(f"\n📊 Chunks oszlopok: {list(chunks_df.columns)}")
print(f"\n🔍 Minta chunk:")
sample_chunk = chunks_df.iloc[0]
print(f"  chunk_id: {sample_chunk.get('chunk_id', 'N/A')}")
print(f"  doc_id: {sample_chunk.get('doc_id', 'N/A')}")
print(f"  text length: {len(str(sample_chunk.get('text', '')))} karakter")
print(f"  court: {sample_chunk.get('court', 'N/A')}")
print(f"  domain: {sample_chunk.get('domain', 'N/A')}")


In [ ]:
# === BM25S BETÖLTÉSE ===
print("🔧 BM25S index betöltése...")
retriever_bm25 = bm25s.BM25.load(str(BM25_INDEX_PATH), mmap=True)

# Chunk IDs betöltése - kezeli JSON és JSONL formátumot is
print(f"   Chunk IDs betöltése: {CHUNK_IDS_PATH}")
try:
    # Először próbáljuk JSON-ként (egyetlen objektum/tömb)
    with open(CHUNK_IDS_PATH, "r", encoding="utf-8") as f:
        content = f.read().strip()
        if content.startswith("[") or content.startswith("{"):
            # JSON formátum
            bm25_chunk_ids = json.loads(content)
        else:
            # JSONL formátum - soronként betöltés
            f.seek(0)
            bm25_chunk_ids = []
            for line in f:
                line = line.strip()
                if line:
                    bm25_chunk_ids.append(json.loads(line))
except json.JSONDecodeError as e:
    # Ha JSON dekódolás sikertelen, próbáljuk JSONL-ként
    print(f"   ⚠️ JSON dekódolás sikertelen, JSONL-ként próbáljuk...")
    bm25_chunk_ids = []
    with open(CHUNK_IDS_PATH, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if line:
                try:
                    bm25_chunk_ids.append(json.loads(line))
                except json.JSONDecodeError:
                    continue

# Ha lista, akkor rendben van; ha dict, akkor értékeket vesszük
if isinstance(bm25_chunk_ids, dict):
    bm25_chunk_ids = list(bm25_chunk_ids.values())

print(f"✅ BM25S betöltve: {len(bm25_chunk_ids):,} chunk")


In [ ]:
# === FAISS BETÖLTÉSE ===
print("\n🔧 FAISS index betöltése...")
index_faiss = faiss.read_index(str(FAISS_INDEX_PATH))

# Chunk ID mapping
chunk_id_map_raw = np.load(CHUNK_ID_MAP_PATH, allow_pickle=True)

# Support both array and dict formats
if isinstance(chunk_id_map_raw, np.ndarray) and chunk_id_map_raw.dtype == object:
    if len(chunk_id_map_raw) > 0 and isinstance(chunk_id_map_raw[0], dict):
        # Dict format: {0: 'chunk_id', 1: 'chunk_id', ...}
        chunk_id_map = chunk_id_map_raw[0]
    else:
        # Array format: ['chunk_id', 'chunk_id', ...]
        chunk_id_map = {i: cid for i, cid in enumerate(chunk_id_map_raw)}
elif isinstance(chunk_id_map_raw, dict):
    chunk_id_map = chunk_id_map_raw
else:
    chunk_id_map = {i: cid for i, cid in enumerate(chunk_id_map_raw)}

print(f"✅ FAISS betöltve: {index_faiss.ntotal:,} vektor")
print(f"   Index típus: {type(index_faiss).__name__}")
print(f"   Chunk ID map: {len(chunk_id_map):,} mapping")


In [ ]:
# === EMBEDDINGGEMMA BETÖLTÉSE ===
print("\n🔧 EmbeddingGemma modell betöltése...")
embedding_model = SentenceTransformer("google/embeddinggemma-300m")

# Move to GPU if available
if torch.cuda.is_available():
    embedding_model = embedding_model.to('cuda')
    print("✅ EmbeddingGemma GPU-n")
else:
    print("✅ EmbeddingGemma CPU-n")

print(f"   Max seq length: {embedding_model.max_seq_length}")
print(f"   Embedding dim: {embedding_model.get_sentence_embedding_dimension()}")


In [ ]:
# === RRF FUSION FÜGGVÉNY ===
def rrf_fusion(bm25_results: List[Tuple[str, float]], 
               faiss_results: List[Tuple[str, float]], 
               k: int = 60) -> List[Tuple[str, float]]:
    """
    Reciprocal Rank Fusion - robusztus, paraméter-mentes fusion.
    
    RRF(d) = Σ 1/(k + rank(d))
    """
    scores = defaultdict(float)
    
    # BM25 ranks
    for rank, (doc_id, _) in enumerate(bm25_results, start=1):
        scores[doc_id] += 1.0 / (k + rank)
    
    # FAISS ranks
    for rank, (doc_id, _) in enumerate(faiss_results, start=1):
        scores[doc_id] += 1.0 / (k + rank)
    
    # Sort by RRF score
    sorted_results = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    return sorted_results

# === CHUNK → DOC AGGREGÁCIÓ ===
def aggregate_chunks_to_docs(chunk_scores: List[Tuple[str, float]]) -> List[Tuple[str, float]]:
    """
    Chunk-level scores → doc-level aggregation (max score).
    chunk_id: "doc_0" → doc_id: "doc"
    """
    doc_scores = {}
    for chunk_id, score in chunk_scores:
        # Strip _N suffix to get doc_id
        doc_id = "_".join(chunk_id.split("_")[:-1]) if "_" in chunk_id else chunk_id
        if doc_id not in doc_scores or score > doc_scores[doc_id]:
            doc_scores[doc_id] = score
    
    sorted_docs = sorted(doc_scores.items(), key=lambda x: x[1], reverse=True)
    return sorted_docs

print("✅ RRF és aggregáció függvények definiálva")


In [ ]:
# === SLATE GENERATION ===
def generate_slate_for_query(query: str, 
                             retriever_bm25, 
                             index_faiss, 
                             embedding_model,
                             chunks_df: pd.DataFrame,
                             chunk_id_map: Dict,
                             bm25_chunk_ids: List[str],
                             qrels_dict: Dict,
                             top_k: int = 30) -> List[Dict]:
    """
    Slate generálás hybrid retrieval-ből (BM25 + FAISS + RRF fusion).
    
    Returns:
        List of dicts with chunk_id, doc_id, text, metadata, scores, relevance
    """
    # === BM25 Retrieval ===
    query_normalized = query.lower().strip()
    query_tokens = bm25s.tokenize(query_normalized, stopwords=None)
    
    bm25_results, bm25_scores = retriever_bm25.retrieve(
        query_tokens, 
        k=TOP_K_BASELINE
    )
    
    # Extract chunk_ids from BM25 corpus structure
    bm25_chunk_scores = []
    for idx in range(len(bm25_results[0])):
        doc_dict = bm25_results[0][idx]
        if isinstance(doc_dict, dict) and 'text' in doc_dict:
            chunk_id = doc_dict['text']
            score = float(bm25_scores[0][idx])
            bm25_chunk_scores.append((chunk_id, score))
    
    # Aggregate to doc level
    bm25_doc_scores = aggregate_chunks_to_docs(bm25_chunk_scores)
    
    # === FAISS Retrieval ===
    query_embedding = embedding_model.encode(
        [query], 
        prompt_name="query",
        normalize_embeddings=True,
        convert_to_numpy=True
    )
    
    # Search FAISS
    distances, indices = index_faiss.search(query_embedding.astype('float32'), TOP_K_BASELINE)
    
    # Map indices to chunk_ids
    faiss_chunk_scores = []
    for dist, idx in zip(distances[0], indices[0]):
        if idx == -1:
            continue
        chunk_id = chunk_id_map.get(int(idx))
        if chunk_id:
            faiss_chunk_scores.append((chunk_id, float(dist)))
    
    # Aggregate to doc level
    faiss_doc_scores = aggregate_chunks_to_docs(faiss_chunk_scores)
    
    # === RRF Fusion ===
    fused_doc_scores = rrf_fusion(bm25_doc_scores, faiss_doc_scores, k=RRF_K)
    top_doc_ids = [doc_id for doc_id, _ in fused_doc_scores[:top_k]]
    
    # === Slate Assembly ===
    slate = []
    for doc_id in top_doc_ids:
        # Find best chunk for this doc (highest BM25 or FAISS score)
        doc_chunks = chunks_df[chunks_df['doc_id'] == doc_id]
        if doc_chunks.empty:
            continue
        
        # Take first chunk (or could be best scoring chunk)
        chunk = doc_chunks.iloc[0]
        
        # Get scores
        bm25_score = next((s for cid, s in bm25_chunk_scores if cid == chunk['chunk_id']), 0.0)
        faiss_score = next((s for cid, s in faiss_chunk_scores if cid == chunk['chunk_id']), 0.0)
        
        # 🔒 CRITICAL FIX: Relevance stored for POST-HOC metrics calculation ONLY!
        # This value is NEVER shown to the model in the evaluation prompt.
        # The model must rank blindly based on content, not ground truth.
        relevance = qrels_dict.get(query, {}).get(doc_id, 0)
        
        slate.append({
            'chunk_id': chunk['chunk_id'],
            'doc_id': doc_id,
            'text': str(chunk.get('text', ''))[:800],  # Truncate to 800 chars
            'court': chunk.get('court', 'N/A'),
            'domain': chunk.get('domain', 'N/A'),
            'year': chunk.get('year', 'N/A'),
            'bm25_score': bm25_score,
            'faiss_score': faiss_score,
            'relevance': relevance,
        })
    
    return slate[:top_k]

print("✅ Slate generation függvény definiálva")


In [ ]:
# === GPU MEMÓRIA TISZTÍTÁSA ===
import gc

if torch.cuda.is_available():
    torch.cuda.empty_cache()
    gc.collect()
    print("✅ CUDA cache törölve")
    
    total = torch.cuda.get_device_properties(0).total_memory / 1e9
    allocated = torch.cuda.memory_allocated(0) / 1e9
    reserved = torch.cuda.memory_reserved(0) / 1e9
    free = total - allocated
    
    print(f"\n📊 GPU Memória Státusz:")
    print(f"  Total: {total:.2f} GB")
    print(f"  Allocated: {allocated:.2f} GB")
    print(f"  Reserved: {reserved:.2f} GB")
    print(f"  Free: {free:.2f} GB")


In [ ]:
# === GRPO MODEL BETÖLTÉSE ===
# A checkpoint-562 mappa csak LoRA adapter fájlokat tartalmaz
# Először betöltjük a base modellt, majd rárakjuk az adaptert
BASE_MODEL_NAME = "unsloth/qwen3-4b-instruct-2507"

# Biztosítjuk, hogy MODEL_PATH Path objektum legyen
if isinstance(MODEL_PATH, str):
    model_path_obj = Path(MODEL_PATH)
else:
    model_path_obj = MODEL_PATH

ADAPTER_PATH = model_path_obj / "checkpoint-562"

print("\n🔄 GRPO model betöltése Unsloth-tal...")
print(f"  Base model: {BASE_MODEL_NAME}")
print(f"  Adapter path: {ADAPTER_PATH}")
print(f"  Max sequence length: {MAX_SEQ_LENGTH}")

# Először betöltjük a base modellt
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=BASE_MODEL_NAME,
    max_seq_length=MAX_SEQ_LENGTH,
    load_in_4bit=True,
    dtype=None,
    device_map="auto",
)

# Rárakjuk a LoRA adaptert
from peft import PeftModel
model = PeftModel.from_pretrained(model, str(ADAPTER_PATH))

model.eval()  # Inference mode
tokenizer.padding_side = "right"

print("\n✅ GRPO model betöltve")
print(f"  Padding side: {tokenizer.padding_side}")
print(f"  Device: {next(model.parameters()).device}")

if torch.cuda.is_available():
    allocated = torch.cuda.memory_allocated(0) / 1e9
    print(f"  GPU memória használat: {allocated:.2f} GB")


In [ ]:
# === TRAIN QUERY IDs MEGHATÁROZÁSA ===
# Az összes qrels query-ből kivonva az eval split query-ket
all_query_ids = set(qrels_dict.keys())
eval_query_ids_set = set(eval_query_ids)
train_query_ids = sorted(list(all_query_ids - eval_query_ids_set))

print(f"\n📊 Query split:")
print(f"  Eval queries: {len(eval_query_ids)}")
print(f"  Train queries: {len(train_query_ids)}")
print(f"  Total queries: {len(all_query_ids)}")


In [ ]:
# === EVALUATION FÜGGVÉNY ===
def evaluate_query(query_id: str,
                   model,
                   tokenizer,
                   retriever_bm25,
                   index_faiss,
                   embedding_model,
                   chunks_df,
                   chunk_id_map,
                   bm25_chunk_ids,
                   qrels_dict,
                   baseline_metrics_dict) -> Optional[Dict]:
    """
    Egyetlen query értékelése:
    1. Slate generation
    2. Prompt creation
    3. Model inference
    4. Parsing
    5. Metric calculation
    """
    try:
        # 1. Generate slate
        # 🔒 Note: qrels_dict passed only to store relevance for metrics, NOT for prompt!
        slate = generate_slate_for_query(
            query_id,
            retriever_bm25,
            index_faiss,
            embedding_model,
            chunks_df,
            chunk_id_map,
            bm25_chunk_ids,
            qrels_dict,
            top_k=SLATE_SIZE
        )
        
        if len(slate) == 0:
            return None
        
        # 2. Create BLIND evaluation prompt (no relevance labels!)
        prompt = create_evaluation_prompt(query_id, slate)
        
        # 3. Model inference
        inputs = tokenizer(
            prompt,
            return_tensors="pt",
            truncation=True,
            max_length=MAX_SEQ_LENGTH
        )
        inputs = {k: v.to(model.device) for k, v in inputs.items()}
        
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=MAX_NEW_TOKENS,
                do_sample=True,
                temperature=TEMPERATURE,
                top_p=TOP_P,
                pad_token_id=tokenizer.pad_token_id,
            )
        
        completion = tokenizer.decode(
            outputs[0][inputs['input_ids'].shape[1]:],
            skip_special_tokens=True
        )
        
        # 4. Parse ranking
        policy_indices = parse_model_ranking(completion, len(slate))
        
        # 5. Get doc IDs
        policy_doc_ids = [slate[i]["doc_id"] for i in policy_indices]
        baseline_doc_ids = [slate[i]["doc_id"] for i in range(len(slate))]
        
        # 6. Compute metrics with ranx
        if query_id not in qrels_dict:
            return None
        
        qrels_ranx = Qrels({query_id: qrels_dict[query_id]})
        
        # Policy run
        policy_run_dict = {
            query_id: {
                doc_id: len(policy_doc_ids) - i
                for i, doc_id in enumerate(policy_doc_ids)
            }
        }
        
        # ⚠️ TRUNCATE TO TOP-K FOR FAIR COMPARISON WITH BASELINE
        # Baseline evaluates on top-20, so we truncate GRPO policy results to top-20
        policy_run_dict_truncated = {
            query_id: dict(
                sorted(
                    policy_run_dict[query_id].items(),
                    key=lambda x: x[1],
                    reverse=True
                )[:EVAL_TOP_K]
            )
        }
        policy_run = Run(policy_run_dict_truncated)
        
        # Compute policy metrics
        metrics_list = [
            "map", "mrr",
            "ndcg@5", "ndcg@10", "ndcg@20",
            "precision@5", "precision@10", "precision@20",
            "recall@5", "recall@10", "recall@20"
        ]
        
        policy_metrics = evaluate(qrels_ranx, policy_run, metrics_list)
        
        # Get baseline metrics
        baseline_metrics = baseline_metrics_dict.get(query_id, {})
        
        # Assemble result
        result = {"query_id": query_id}
        
        for metric in metrics_list:
            baseline_key = metric.replace("@", "@")
            result[f"baseline_{baseline_key}"] = baseline_metrics.get(baseline_key, 0.0)
            result[f"policy_{baseline_key}"] = policy_metrics.get(metric, 0.0)
        
        # Improvement

        result["improvement_ndcg@10"] = (

            result["policy_ndcg@10"] - result["baseline_ndcg@10"]print("✅ Evaluation függvény definiálva")

        )

                return None

        return result        print(f"\n⚠️ Hiba query-nél ({query_id[:50]}...): {e}")

            except Exception as e:

In [ ]:
# === EVALUATION LOOP - EVAL SPLIT ===
print("\n" + "="*80)
print("🚀 EVAL SPLIT ÉRTÉKELÉSE")
print("="*80)

eval_results = []

for query_id in tqdm(eval_query_ids, desc="Eval queries"):
    result = evaluate_query(
        query_id,
        model,
        tokenizer,
        retriever_bm25,
        index_faiss,
        embedding_model,
        chunks_df,
        chunk_id_map,
        bm25_chunk_ids,
        qrels_dict,
        baseline_metrics_dict
    )
    
    if result:
        eval_results.append(result)

print(f"\n✅ Eval értékelés kész: {len(eval_results)}/{len(eval_query_ids)} sikeres")


In [ ]:
# === EVALUATION LOOP - TRAIN SPLIT ===
print("\n" + "="*80)
print("🚀 TRAIN SPLIT ÉRTÉKELÉSE")
print("="*80)

train_results = []

for query_id in tqdm(train_query_ids, desc="Train queries"):
    result = evaluate_query(
        query_id,
        model,
        tokenizer,
        retriever_bm25,
        index_faiss,
        embedding_model,
        chunks_df,
        chunk_id_map,
        bm25_chunk_ids,
        qrels_dict,
        baseline_metrics_dict
    )
    
    if result:
        train_results.append(result)

print(f"\n✅ Train értékelés kész: {len(train_results)}/{len(train_query_ids)} sikeres")


In [ ]:
# === EVAL SPLIT EXPORT ===
eval_df = pd.DataFrame(eval_results)
eval_output_path = OUTPUT_DIR / "eval_per_query_results.csv"
eval_df.to_csv(eval_output_path, index=False, encoding='utf-8')

print(f"\n✅ Eval results exportálva: {eval_output_path}")
print(f"   {len(eval_df)} query × {len(eval_df.columns)} oszlop")

# === TRAIN SPLIT EXPORT ===
train_df = pd.DataFrame(train_results)
train_output_path = OUTPUT_DIR / "train_per_query_results.csv"
train_df.to_csv(train_output_path, index=False, encoding='utf-8')

print(f"\n✅ Train results exportálva: {train_output_path}")
print(f"   {len(train_df)} query × {len(train_df.columns)} oszlop")


In [ ]:
# === SUMMARY STATISTICS ===
print("\n" + "="*80)
print("📊 EVALUATION SUMMARY")
print("="*80)

# Eval split summary
if len(eval_df) > 0:
    print("\n🎯 EVAL SPLIT:")
    print(f"  Query count: {len(eval_df)}")
    
    for metric in ['ndcg@10', 'ndcg@5', 'map', 'mrr']:
        baseline_mean = eval_df[f'baseline_{metric}'].mean()
        policy_mean = eval_df[f'policy_{metric}'].mean()
        improvement = policy_mean - baseline_mean
        relative = (improvement / baseline_mean * 100) if baseline_mean > 0 else 0
        
        print(f"\n  {metric.upper()}:")
        print(f"    Baseline: {baseline_mean:.4f}")
        print(f"    Policy:   {policy_mean:.4f}")
        print(f"    Δ:        {improvement:+.4f} ({relative:+.1f}%)")

# Train split summary
if len(train_df) > 0:
    print("\n\n🎯 TRAIN SPLIT:")
    print(f"  Query count: {len(train_df)}")
    
    for metric in ['ndcg@10', 'ndcg@5', 'map', 'mrr']:
        baseline_mean = train_df[f'baseline_{metric}'].mean()
        policy_mean = train_df[f'policy_{metric}'].mean()
        improvement = policy_mean - baseline_mean
        relative = (improvement / baseline_mean * 100) if baseline_mean > 0 else 0
        
        print(f"\n  {metric.upper()}:")
        print(f"    Baseline: {baseline_mean:.4f}")
        print(f"    Policy:   {policy_mean:.4f}")
        print(f"    Δ:        {improvement:+.4f} ({relative:+.1f}%)")


In [ ]:
# === SUMMARY JSON EXPORT ===
summary = {
    "eval_split": {
        "query_count": len(eval_df),
        "metrics": {}
    },
    "train_split": {
        "query_count": len(train_df),
        "metrics": {}
    }
}

# Eval metrics
if len(eval_df) > 0:
    for metric in ['ndcg@10', 'ndcg@5', 'map', 'mrr', 'recall@10']:
        summary["eval_split"]["metrics"][metric] = {
            "baseline": float(eval_df[f'baseline_{metric}'].mean()),
            "policy": float(eval_df[f'policy_{metric}'].mean()),
            "improvement": float(eval_df[f'policy_{metric}'].mean() - eval_df[f'baseline_{metric}'].mean())
        }

# Train metrics
if len(train_df) > 0:
    for metric in ['ndcg@10', 'ndcg@5', 'map', 'mrr', 'recall@10']:
        summary["train_split"]["metrics"][metric] = {
            "baseline": float(train_df[f'baseline_{metric}'].mean()),
            "policy": float(train_df[f'policy_{metric}'].mean()),
            "improvement": float(train_df[f'policy_{metric}'].mean() - train_df[f'baseline_{metric}'].mean())
        }

summary_path = OUTPUT_DIR / "evaluation_summary.json"
with open(summary_path, 'w', encoding='utf-8') as f:
    json.dump(summary, f, ensure_ascii=False, indent=2)

print(f"\n✅ Summary JSON exportálva: {summary_path}")


In [ ]:
# === SAMPLE RESULTS ===
print("\n" + "="*80)
print("📋 MINTA EREDMÉNYEK (Eval Split)")
print("="*80)

if len(eval_df) > 0:
    # Top 5 legnagyobb javulás
    top_improvements = eval_df.nlargest(5, 'improvement_ndcg@10')
    
    print("\n🏆 Top 5 legnagyobb javulás (nDCG@10):")
    for idx, row in top_improvements.iterrows():
        print(f"\n  Query: {row['query_id'][:60]}...")
        print(f"    Baseline nDCG@10: {row['baseline_ndcg@10']:.4f}")
        print(f"    Policy nDCG@10:   {row['policy_ndcg@10']:.4f}")
        print(f"    Improvement:      {row['improvement_ndcg@10']:+.4f}")
    
    # Top 5 legnagyobb romlás
    print("\n\n⚠️ Top 5 legnagyobb romlás (nDCG@10):")
    worst_improvements = eval_df.nsmallest(5, 'improvement_ndcg@10')
    
    for idx, row in worst_improvements.iterrows():
        print(f"\n  Query: {row['query_id'][:60]}...")
        print(f"    Baseline nDCG@10: {row['baseline_ndcg@10']:.4f}")
        print(f"    Policy nDCG@10:   {row['policy_ndcg@10']:.4f}")
        print(f"    Improvement:      {row['improvement_ndcg@10']:+.4f}")

print("\n" + "="*80)
print("✅ EVALUATION KÉSZ!")
print("="*80)
print(f"\n📁 Kimeneti fájlok:")
print(f"  • {eval_output_path}")
print(f"  • {train_output_path}")
print(f"  • {summary_path}")
print("\n💡 Töltsd le ezeket a fájlokat lokálisan a data/models/grpo_policy/ mappába!")


In [ ]:
# === ADATSZIVÁRGÁS VALIDÁCIÓ ===
print("\n" + "="*80)
print("🔒 ADATSZIVÁRGÁS ELLENŐRZÉS")
print("="*80)

# 1. Promptban nincs relevancia címke?
print("\n1️⃣ Prompt validáció:")
test_query = eval_query_ids[0] if len(eval_query_ids) > 0 else list(qrels_dict.keys())[0]
test_slate = generate_slate_for_query(
    test_query,
    retriever_bm25,
    index_faiss,
    embedding_model,
    chunks_df,
    chunk_id_map,
    bm25_chunk_ids,
    qrels_dict,
    top_k=5
)
test_prompt = create_evaluation_prompt(test_query, test_slate)

# Ellenőrzés: nincs-e "Relevance:" vagy "Grade" a promptban
has_relevance_leak = "Relevance:" in test_prompt or "Grade" in test_prompt
if has_relevance_leak:
    print("   ❌ HIBA: Relevancia címke BENNE VAN a promptban!")
    print("   ⚠️  Ez adatszivárgást okoz!")
else:
    print("   ✅ PASS: Nincs relevancia címke a promptban")

# 2. Slate tartalmaz relevancia mezőt? (csak metrikákhoz)
if len(test_slate) > 0:
    has_relevance_field = 'relevance' in test_slate[0]
    if has_relevance_field:
        print("\n2️⃣ Slate tartalmaz relevancia mezőt:")
        print("   ✅ PASS: relevance mező van (metrikákhoz kell)")
        print("   🔒 Ez rendben van, AMÍG nem kerül a promptba!")
    else:
        print("\n2️⃣ Slate NEM tartalmaz relevancia mezőt:")
        print("   ❌ HIBA: relevance mező hiányzik a slate-ből!")

# 3. Eval/Train split tisztasága
print("\n3️⃣ Eval/Train split validáció:")
eval_set = set(eval_query_ids)
train_set = set(train_query_ids)
overlap = eval_set & train_set

if len(overlap) == 0:
    print(f"   ✅ PASS: Nincs kontamináció (0 átfedés)")
    print(f"   • Eval split: {len(eval_set)} query")
    print(f"   • Train split: {len(train_set)} query")
else:
    print(f"   ❌ HIBA: {len(overlap)} query átfedés van!")

# Összegzés
print("\n" + "="*80)
print("📊 VALIDÁCIÓ ÖSSZEGZÉS")
print("="*80)

all_pass = not has_relevance_leak and has_relevance_field and len(overlap) == 0

if all_pass:
    print("✅ MINDEN ELLENŐRZÉS SIKERES!")
    print("   • Prompt tiszta (nincs adatszivárgás)")
    print("   • Slate struktúra helyes")
    print("   • Nincs kontamináció eval/train között")
    print("\n💡 Az eredmények megbízhatók és interpretálhatók!")
else:
    print("❌ VALIDÁCIÓ SIKERTELEN!")
    if has_relevance_leak:
        print("   • Relevancia címke BENNE VAN a promptban")
    if not has_relevance_field:
        print("   • Relevancia mező HIÁNYZIK a slate-ből")
    if len(overlap) > 0:
        print(f"   • {len(overlap)} query kontamináció")
    print("\n⚠️  NE használd ezeket az eredményeket, javítsd a notebook-ot!")
